In [20]:
import os
import glob
import pickle
import multiprocessing
from beautifultable import BeautifulTable as BT

# Switching multiprocessing to avoid "Too many files opened"
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler


import sys
sys.path.append('..')
sys.path.append('ResNets')
from utils import load_dataset, count_parameters


import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', 'ImportWarning')
warnings.filterwarnings('ignore', 'DeprecationWarning')

In [21]:
dataset = 'CIFAR10'
testing = False             # Activate test to run few iterations per epoch       
comments = True             # Activate printing comments
ensemble_type = 'Big'       # Single model big 
#ensemble_type = 'Huge'     # Single model huge
batch_size = 128
n_epochs = 300
n_iters = 64000
learning_rate = 0.001

momentum = 0.9
weight_decay = 1e-4

# GPU if CUDA is available
cuda = torch.cuda.is_available()
n_workers = multiprocessing.cpu_count()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
gpus = True if torch.cuda.device_count() > 1 else False
mem = False if device == 'cpu' else True


In [22]:
scripts = os.getcwd()
root = os.path.abspath(os.path.join(scripts, '../'))
results = os.path.abspath(os.path.join(root, 'results'))
data_path = os.path.abspath(os.path.join(root, '../datasets'))

path_to_logs = os.path.join(results, 'logs', 'resnets')
path_to_models = os.path.join(results, 'models', 'resnets')
path_to_figures = os.path.join(results, 'figures', 'resnets')
path_to_definitives = os.path.join(path_to_models, 'definitives')

train_log = os.path.join(path_to_logs, 'train')
test_log = os.path.join(path_to_logs, 'test')

assert os.path.exists(root), 'Root folder not found'
assert os.path.exists(scripts), 'Scripts folder not found'
assert os.path.exists(results), 'Results folder not found'
assert os.path.exists(data_path), 'Data folder not found'
assert os.path.exists(path_to_logs), 'Logs folder not found'
assert os.path.exists(path_to_models), 'Models folder not found'
assert os.path.exists(path_to_figures), 'Figure folder not found'
assert os.path.exists(path_to_definitives), 'Def. models folder not found'

paths = {
    'root': root, 
    'script': scripts,
    'data': data_path,
    'resulsts': results,
    'logs': {'train': train_log, 'test': test_log}, 
    'models': path_to_models,
    'definitives': path_to_definitives,
    'figures': path_to_figures
}

In [23]:
dataset = 'CIFAR10'
comments=True
train_set, valid_set, test_set = load_dataset(data_path, dataset, comments=comments)

train_loader = DataLoader(dataset = train_set.dataset, 
                          sampler=SubsetRandomSampler(train_set.indices),
                          batch_size = batch_size, num_workers=n_workers,
                          pin_memory = mem)

valid_loader = DataLoader(dataset = valid_set.dataset, 
                          sampler=SubsetRandomSampler(valid_set.indices),
                          batch_size = batch_size, num_workers=n_workers,
                          pin_memory = mem)

test_loader = DataLoader(dataset = test_set, batch_size = 128,
                         shuffle = False, num_workers=n_workers, pin_memory = mem)

Files already downloaded and verified
Loading dataset:  CIFAR10
+--------------+-------+
| Train Images | 45000 |
+--------------+-------+
| Valid Images | 5000  |
+--------------+-------+
| Test Images  | 10000 |
+--------------+-------+
|   Classes    |  10   |
+--------------+-------+


## Accuracy on the last trained model on Test Set

In [8]:
from resnets_Paper import ResNet20, ResNet32, ResNet44, ResNet56, ResNet110

model = ResNet56()
model.load_state_dict(torch.load('../ResNet56-156.pkl', map_location=device))
model.eval()
model.to(device)
control = 0

total, correct = (0,0)
from torch.autograd import Variable
with torch.no_grad():
    
    for i, (images, labels) in enumerate(test_loader):
        
        images = Variable(images)
        labels = Variable(labels)
        
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        
        _, preds = outputs.max(1)
        total += outputs.size(0)
        correct += int(sum(preds == labels))
        
        control += 1
    
print('Single model accuracy {}%'.format(100 * correct / total))
print('Control: ', control)

Single model accuracy 90.21%
Control:  10000


## Accuracy on the last saved model on Test Set

In [14]:
from collections import OrderedDict

singleModel = ResNet56()
ensemble_size = 'Big'

def get_epoch(pth):
    pth = pth.split('/')[-1] # remove all the path
    pth = pth[:-4] # remove .pkl
    epoch = pth.split('-')[1] # get just the epoch
    print('Epoch to restart training: ', epoch)
    return epoch

def load_weights(path, verbose=0):
    global device
    print('\n\nLoading Weights from: ', path)
    print('-------')
    state_dict = torch.load(path, map_location=device)
    if verbose == 1: print('\nCurrent dict: ', state_dict.keys())
    new_state_dict = OrderedDict()
    for k,v in state_dict.items():
        name = k[7:] # remove module.
        new_state_dict[name] = v
    if verbose == 1: print('\nNew dict: ', new_state_dict.keys())
    return new_state_dict

## LOAD TRAINED MODELS      -->         args.pretrained = -P = True
print('Loading trained models... ')

# Load saved models
if ensemble_type == 'Big':
    pth = os.path.join(path_to_definitives, 'resnet56')
else:
    pth = os.path.join(path_to_definitives, 'resnet110')
    e_epoch = get_epoch(pth)

assert os.path.exists(pth), 'Model to load not found'
ps = glob.glob(os.path.join(pth, '*.pkl'))
print('Files to load:')
for p in ps:
    print('...', p[-25:])

pattern = 'ResNet56' if ensemble_size == 'Big' else 'ResNet110'         
single_ps = [p for p in ps if 'ResNet56' in p]
ensemble_ps = [p for p in ps if 'ResNet56' not in p]


# Single Model
print('Getting ready Single Model : ', singleModel)
s_epoch = int(get_epoch(single_ps[0]))
singleModel.load_state_dict(load_weights(single_ps[0], verbose=0))
print('[OK] Single model loaded on epoch ', s_epoch)

Loading trained models... 
Files to load:
... snet56/ResNet20_3-149.pkl
... snet56/ResNet20_2-149.pkl
... resnet56/ResNet56-161.pkl
... snet56/ResNet20_1-149.pkl
Getting ready Single Model :  ResNet(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(16, 16, kernel

In [18]:
from resnets_Paper import ResNet20, ResNet32, ResNet44, ResNet56, ResNet110

singleModel.eval()
singleModel.to(device)
control = 0

total, correct = (0,0)
from torch.autograd import Variable
with torch.no_grad():
    
    for i, (images, labels) in enumerate(test_loader):
        
        images = Variable(images)
        labels = Variable(labels)
        
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = singleModel(images)
        
        _, preds = outputs.max(1)
        total += outputs.size(0)
        correct += int(sum(preds == labels))
        
        control += test_loader.batch_size
    
print('Single model accuracy {}%'.format(100 * correct / total))
print('Control: ', control)

Single model accuracy 89.43%
Control:  10112


## Accuracy on the Validation Set

In [25]:
control = 0
total, correct = (0,0)
from torch.autograd import Variable
with torch.no_grad():
    
    for i, (images, labels) in enumerate(valid_loader):
        
        images = Variable(images)
        labels = Variable(labels)
        
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = singleModel(images)
        
        _, preds = outputs.max(1)
        total += outputs.size(0)
        correct += int(sum(preds == labels))
        
        control += valid_loader.batch_size
    
print('Single model accuracy {}%'.format(100 * correct / total))
print('Control: ', control)

Single model accuracy 98.66%
Control:  5120
